In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import sys
import re
import os
import warnings

from scipy.optimize import least_squares
import scipy.linalg as spL

from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

warnings.filterwarnings('ignore')

from importlib import reload

In [35]:
# Import the neccessary modules

Module_path = '../Modules'  #specify path to modules folder, change as needed for environment


sys.path.insert(0,Module_path)



import base_functions as baf
reload(baf)
import solution_analysis_utils as sa
reload(sa)

<module 'solution_analysis_utils' from '/Users/king339/Projects/Predictive_Phenomics/Code/Bolt/01_Glycolosis_Example/../Modules/solution_analysis_utils.py'>

## Import a Solution

In [36]:
import pickle




saved_solution_file = 'max_flux_glyc_exmp_solution.pkl'


model_file = 'glycolosis_tca_ppp_prot4.pkl'


#Load the model file
S_active,active_reactions, metabolites, Keq= baf.load_model(model_file)


#load the solution file
y_sol,alpha_sol,n_sol,S,Keq,f_log_counts,vcount_upper_bound,obj_coefs = baf.load_model_solution(saved_solution_file)


# Computing Equivalent Solutions

Only a subset of the reactions and metabolites are restricted at solutions. There is a subset of metabolites and fluxes that can be varied. 




## Identify the Fixed and Free Metabolites and Reactions

### Find the restricted metabolites and reactions

Find the gradient direction


We can understand the objective maximize sum(y) 

as instead the objective maximize_(beta) ( gamma_1\*beta_1 + gamma_2\*beta_2 ...)
and compute gradient direction with respect to beta

In [37]:

### compute decompositions of the stochiometric matrix
S_v, S_f, Sv_N = baf.reduced_Stoich(S,f_log_counts,vcount_upper_bound)


## find the gradient direction projected into the steady state subspace
obj_coefs = np.reshape(np.ravel(obj_coefs),(1,len(obj_coefs)))
beta_grad = np.matmul(obj_coefs,Sv_N)
beta_grad = np.transpose(beta_grad)

In [38]:

### Find the consistency equations that must be active at solutions

restricted_reactions = sa.obj_restricted_rxns(beta_grad,y_sol,n_sol,f_log_counts,Sv_N,S_v,S_f,Keq)


#compute basis for allowable free metabolite changes

N_Svr = sa.free_metabolite_basis(restricted_reactions,S_v)

## Equivalent Solutions with Maximum and Minimum Total Metabolite Counts

In [39]:

max_n_tot, max_n_alpha_tot = sa.max_min_total_metabolite_sol(y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'max') 

min_n_tot, min_n_alpha_tot = sa.max_min_total_metabolite_sol(y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'min') 

## Equivalent Solutions with Maximum and Minimum Toatal Activity Coefficients

In [40]:

max_n_ac_tot, max_ac_alpha_tot = sa.max_min_total_activity_sol(y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'max') 

min_n_ac_tot, min_ac_alpha_tot = sa.max_min_total_activity_sol(y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'min') 

## For Each Metabolite Identify the Maximum and Minimum Possible Counts

In [41]:
Metabolite_sol_summary = metabolites[metabolites['Variable'] == True]
n_react = S.shape[0]
n_M_v = len(vcount_upper_bound)
metabolite_indexes = np.arange(0,n_react,1)

max_n_list = []
min_n_list = []
for i in range(n_M_v):
    metab_index = np.array([i])

    max_n, max_n_alpha = sa.max_min_subset_metabolite_sol(metab_index,y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'max') 

    min_n, min_n_alpha = sa.max_min_subset_metabolite_sol(metab_index,y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'min')

    max_n_list.append(max_n[i])
    min_n_list.append(min_n[i]) 
    

Metabolite_sol_summary['Max Possible'] = max_n_list
Metabolite_sol_summary['Min Possible'] = min_n_list

Metabolite_sol_summary['Min total solution'] = min_n_tot
Metabolite_sol_summary['Max total solution'] = max_n_tot

#display(Metabolite_sol_summary)

### find the metabolites that can vary and print those
display(Metabolite_sol_summary[ np.abs(Metabolite_sol_summary['Max Possible'] - Metabolite_sol_summary['Min Possible'] )> 1e-10  ])


,Conc,Conc_Experimental,Variable,Max Possible,Min Possible,Min total solution,Max total solution
"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",0.001,0.0152,True,12.484126,-10.209962,-10.209962,12.484126


## For Each Reaction Identify the Maximum and Minimum Possible Activity Coefficient Choices

In [42]:

Reaction_sol_summary = active_reactions[['LEFT','RIGHT','Full Rxn']]

reaction_indexes = np.arange(0,n_react,1)

max_ac_list = []
min_ac_list = []
for j in range(n_react):
    rxn_index = np.array([j])

    max_n_ac, max_ac_alpha = sa.max_min_subset_activity_sol(rxn_index,y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'max') 

    min_n_ac, min_ac_alpha = sa.max_min_subset_activity_sol(rxn_index,y_sol,n_sol,N_Svr,vcount_upper_bound,f_log_counts,np.transpose(S),S_v,S_f,Keq,sense = 'min')

    max_ac_list.append(max_ac_alpha[j])
    min_ac_list.append(min_ac_alpha[j]) 
    


Reaction_sol_summary['Solution Flux'] = y_sol
Reaction_sol_summary['Max Possible activity'] = max_ac_list
Reaction_sol_summary['Min Possible activity'] = min_ac_list


Reaction_sol_summary['Max Total Activity'] = max_ac_alpha_tot
Reaction_sol_summary['Min Total Activity'] = min_ac_alpha_tot


#display(Reaction_sol_summary)


### find the reactions where the activity coefficient can vary and print those
display(Reaction_sol_summary[ np.abs(Reaction_sol_summary['Max Possible activity'] - Reaction_sol_summary['Min Possible activity'] )>= 1e-10  ])

,LEFT,RIGHT,Full Rxn,Solution Flux,Max Possible activity,Min Possible activity,Max Total Activity,Min Total Activity
FBA,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL = GLYCERON...",1.102045,9.649060e-08,1.344522e-17,1.344522e-17,1.344522e-17
PFK,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ATP:CYTOSOL,"ADP:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ATP:CYTOSOL =...,1.102045,1.000000e+00,9.071522e-11,1.000000e+00,1.000000e+00
